# BAN CARBON Revenue Forecast

This notebook implements the revenue forecasting methodology described in `/home/fdvom/ban-carbon-hq/research/revenue-forecast/context/forecasting-methodology.md`.

## Overview

The forecast computes M-month grassroots donation revenue based on:
- **Channels**: Ways to reach potential donors
- **Segments**: Donor groups with shared characteristics
- **Funnel parameters**: Reachable audience, lead rates, conversion rates
- **Donor behavior**: Gift amounts and attrition rates

## Revenue Model

```
revenue = Σ_c Σ_s [ n_cs * lead_c * conv * Σ_{t=m_c}^M gift_s * (1 - attr)^{t - m_c} ]
```

In [59]:
import pandas as pd
import numpy as np
from pathlib import Path
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows
from datetime import datetime

## 1. Load Data

In [60]:
# Define paths
data_dir = Path("../../../data/raw/revenue-forecasts")

# Load core tables
channels = pd.read_csv(data_dir / "channels.csv")
segments = pd.read_csv(data_dir / "segments.csv")

# Load lead rates and gift amounts
leads_raw = pd.read_csv(data_dir / "lead-rates/combined.csv")
gifts_raw = pd.read_csv(data_dir / "gift-amounts/combined.csv")

# Average lead rates across LLMs
leads = leads_raw[['id_cha']].copy()
leads['lead'] = leads_raw[['lead_chatgpt', 'lead_claude', 'lead_gemini']].mean(axis=1)

# Average gift amounts across LLMs
gifts = gifts_raw[['id_seg']].copy()
gifts['gift'] = gifts_raw[['gift_chatgpt', 'gift_claude', 'gift_gemini']].mean(axis=1)

# Load audience data (ChatGPT only - most conservative)
audience = pd.read_csv(data_dir / "audience-size/chatgpt.csv")

# Load channel sequencing from all three sources
seq_chatgpt = pd.read_csv(data_dir / "channel-sequence/chatgpt.csv")
seq_claude = pd.read_csv(data_dir / "channel-sequence/claude.csv")
seq_gemini = pd.read_csv(data_dir / "channel-sequence/gemini.csv")

# Merge and average sequences
seq_chatgpt = seq_chatgpt.rename(columns={'month': 'month_chatgpt'})
seq_claude = seq_claude.rename(columns={'month': 'month_claude'})
seq_gemini = seq_gemini.rename(columns={'month': 'month_gemini'})

channel_seq = seq_chatgpt[['id_cha', 'month_chatgpt']].merge(
    seq_claude[['id_cha', 'month_claude']], on='id_cha', how='outer'
).merge(
    seq_gemini[['id_cha', 'month_gemini']], on='id_cha', how='outer'
)
channel_seq['month'] = channel_seq[['month_chatgpt', 'month_claude', 'month_gemini']].mean(axis=1).round().astype(int)
channel_seq = channel_seq[['id_cha', 'month']]

# Load structural parameters
struct_params = pd.read_csv(data_dir / "structural-parameters.csv")
conv = struct_params['conv'].iloc[0]
attr = struct_params['attr'].iloc[0]
M = 12

print(f"✓ Loaded {len(channels)} channels, {len(segments)} segments")
print(f"✓ Parameters: conv={conv:.1%}, attr={attr:.1%}, M={M} months")

✓ Loaded 77 channels, 14 segments
✓ Parameters: conv=12.0%, attr=2.2%, M=12 months


## 2. Calculate Revenue

In [80]:
# Create base table
base = audience.merge(leads[['id_cha', 'lead']], on='id_cha', how='left')
base = base.merge(gifts[['id_seg', 'gift']], on='id_seg', how='left')
base = base.merge(channel_seq[['id_cha', 'month']], on='id_cha', how='left')
base = base.rename(columns={'month': 'm_c', 'lead': 'lead_c', 'gift': 'gift_s', 'n': 'n_cs'})

# Compute number of donors from each channel-segment that are active each month
base["starting_donors"] = base["n_cs"] * base["lead_c"] * conv
cols = ["id_cha", "id_seg", "starting_donors", "gift_s", "m_c"]
base = base[cols]
months = np.tile(np.arange(1, 13), len(base))
base = pd.DataFrame(np.repeat(base.values, M, axis = 0), columns = cols)
base["month"] = months
base["months_active"] = base["month"] - base["m_c"]
base["donors"] = base["starting_donors"] * (1 - attr)**base["months_active"]
base.loc[base["months_active"] < 0, "donors"] = 0

# Compute revenue
base["revenue"] = base["donors"] * base["gift_s"]

# Filter out channels not reached within M
base = base[base['m_c'] <= M]

# Print total revenue
total_revenue = base['revenue'].sum()
print(f"\n✓ Total 12-mo Revenue: ${total_revenue:,.2f}")
print(f"✓ Channel-segment pairs in forecast: {len(base[["id_cha", "id_seg"]].drop_duplicates())}")


✓ Total 12-mo Revenue: $105,179.97
✓ Channel-segment pairs in forecast: 22


## 3. Calculate Monthly Breakdown

In [82]:
# Calculate monthly revenue directly from base
monthly_revenue = base.groupby('month')[['donors', 'revenue']].sum().reset_index()
monthly_revenue.columns = ['month', 'total_donors', 'revenue']
monthly_revenue = monthly_revenue.sort_values('month')
monthly_revenue['cumulative_revenue'] = monthly_revenue['revenue'].cumsum()

print("\n✓ Monthly breakdown calculated")
monthly_revenue


✓ Monthly breakdown calculated


,month,total_donors,revenue,cumulative_revenue
0,1,0,0.0,0.0
1,2,38.72,2984.666667,2984.666667
2,3,51.94816,3599.537333,6584.204
3,4,137.6213,7965.467512,14549.671512
4,5,134.593632,7790.227227,22339.898739
5,6,193.072572,9881.135561,32221.0343
6,7,206.056975,10755.583912,42976.618212
7,8,253.175722,12192.014399,55168.632611
8,9,271.605856,12563.790083,67732.422694
9,10,277.070527,12592.453367,80324.876061


## 4. Calculate Expenses

In [ ]:
#TODO


✓ Total Expenses: $121,339.83
✓ Net Position: $-16,159.85


,month,founder_comp,contract_support,software_infra,accounting_legal,transaction_fees,fiscal_sponsor_fees,marketing,total_expenses,cumulative_expenses
0,1,5000,3000,500,800,0.000000,0.000000,2000,11300.000000,11300.000000
1,2,5000,3000,500,800,100.672000,335.573333,2000,11736.245333,23036.245333
2,3,5000,2000,500,800,118.873216,396.244053,1500,10315.117269,33351.362603
3,4,5000,2000,500,800,276.867605,922.892017,1500,10999.759623,44351.122225
4,5,5000,1500,500,800,270.776518,902.588393,1000,9973.364911,54324.487136
5,6,5000,1500,500,800,318.477035,1061.590115,1000,10180.067150,64504.554286
6,7,5000,1000,500,800,398.779340,1329.264466,800,9828.043806,74332.598092
7,8,5000,1000,500,800,443.930882,1479.769608,800,10023.700490,84356.298582
8,9,5000,500,500,800,453.364403,1511.214676,600,9364.579079,93720.877661
9,10,5000,500,500,800,452.542386,1508.474620,600,9361.017006,103081.894667


## 5. Generate Excel Budget

Create comprehensive 5-tab Excel workbook for fiscal sponsor review.

In [48]:
# [Note: Excel generation code would go here - over 500 lines]
# For now, run the complete script separately
print("\n" + "="*80)
print("To generate the full Excel budget file, run:")
print("  python run_budget_complete.py")
print("="*80)


To generate the full Excel budget file, run:
  python run_budget_complete.py
